#### New to Plotly?
Plotly's Python library is free and open source! [Get started](https://plot.ly/python/getting-started/) by downloading the client and [reading the primer](https://plot.ly/python/getting-started/).
<br>You can set up Plotly to work in [online](https://plot.ly/python/getting-started/#initialization-for-online-plotting) or [offline](https://plot.ly/python/getting-started/#initialization-for-offline-plotting) mode, or in [jupyter notebooks](https://plot.ly/python/getting-started/#start-plotting-online).
<br>We also have a quick-reference [cheatsheet](https://images.plot.ly/plotly-documentation/images/python_cheat_sheet.pdf) (new!) to help you get started!

#### Plotly Dashboards 2.0
Plotly now supports online interactive dashboards. Plotly dashboards can be created with our online [drag-and-drop dashboard creator](https://plot.ly/dashboards/create) or programatically in Python, R, or MATLAB. Dashboards are great for showcasing a collection of graphs like your KPIs, experimental results, or the real-time status of your systems.
<br>
Like Plotly graphs, datasets, and presentations, you can create dashboards entirely programatically with our APIs. This tutorial explains how to create dashboards programatically with Python. No HTML, CSS, or Javascript is required to create these dashboards, it's all pure Python!
<br>
Plotly's dashboards aren't part of the official Plotly python API yet but they will be in the coming months. In the meantime, we'll make requests with the `requests` library to [Plotly's V2 API](https://api.plot.ly/v2/dashboards).

In [23]:
import plotly

import json
import pprint
import requests
import webbrowser

from IPython import display

api_key = plotly.tools.get_credentials_file()['api_key'] 

# Wrapper around requests.get
def get(*args, **kwargs):
  return requests.get(*args, auth=('chelsea_lyn', api_key), headers={'Plotly-Client-Platform': 'nteract'}, **kwargs)

# upload dashboard to your plotly account
def create_dashboard(dashboard_object, filename, world_readable, auto_open=True):
  	# There are a few short-comings to this function. These will get worked out once this is part of the official plotly python library.
    # - filename needs to be able to support `/` to create or use folders. This'll require a few API calls.
    # - this function only works if the filename is unique. Need to call `update` if this file already exists to 
    # 	overwrite the file
    # - world_readable really should be `sharing` and allow `public`, `private`, or `secret` like in `py.plot`
  	# - dashboard content should be validated before sending it off
    # - auto_open parameter for opening the result
    
	res = requests.post(
      'https://api.plot.ly/v2/dashboards',
      auth=('chelsea_lyn', api_key),
      headers={
        'Plotly-Client-Platform': 'nteract'
      },
      data={
        'content': json.dumps(dashboard_object),
        'filename': filename,
        'world_readable': world_readable
      }
    )
	res.raise_for_status()
    
 	url = res.json()['web_url']
	webbrowser.open_new(res.json()['web_url'])
	return url

#### Viewing and downloading dashboards that you have created

Get started by playing around with the [online dashboard editor](https://plot.ly/dashboards/create). Once you've created a few dashboards, we can download the contents with the API.
<br>
You can get a list of the current dashboards by viewing the REST API docs at https://api.plot.ly/v2/dashboards
<br>
You can view your dashboards in your list of files at https://plot.ly/organize/

In [24]:
dashboards = []
response = get('https://api.plot.ly/v2/dashboards').json()
dashboards += [dashboard for dashboard in response['results'] if not dashboard['deleted']]
while response['next']: # paginated
	response = get(response['next']).json()
	dashboards += [dashboard for dashboard in response['results'] if not dashboard['deleted']]

[d['filename'] for d in dashboards]

[u'Tern Plots', u'Dashboard Tutorial - Energy Example']

Similar to Plotly graphs, dashboards are described declarively in an open JSON format.

Let's take a look at a simple dashboard: [https://plot.ly/dashboard/chris:17748/present](https://plot.ly/dashboard/chris:17748/present).
![Simple dashboard example](https://cdn.pbrd.co/images/IAqBYVjj.png)

We can look at the JSON description of this dashboard by downloading it through the API:

In [25]:
# Everything that Plotly stores about the dashboard is in this response
dashboard = get(dashboards[0]['api_urls']['dashboards']).json()

# In particular, view the JSON representation of one 
pprint.pprint(json.loads(dashboard['content']))

{u'layout': {u'direction': u'vertical',
             u'first': {u'direction': u'vertical',
                        u'first': {u'direction': u'vertical',
                                   u'first': {u'direction': u'vertical',
                                              u'first': {u'direction': u'horizontal',
                                                         u'first': {u'boxType': u'plot',
                                                                    u'fileId': u'chelsea_lyn:13091',
                                                                    u'shareKey': u'KIfuEpo259nkweMFWe7NFM',
                                                                    u'type': u'box'},
                                                         u'second': {u'boxType': u'plot',
                                                                     u'fileId': u'chelsea_lyn:13095',
                                                                     u'shareKey': u'T2W34xmiL4QejA88LStF8J',
   

This content completely describes the dashboard. Although we created this dashboard in the online editor, we can re-create it programatically by uploading that same content to Plotly's API.

Here's the same description of the dashboard, annotated with some comments. The full dashboards spec is further below.
<br>
Declaritvely describe the dashboard:

In [26]:
dashboard_1 = {
  'layout': {
    # parent node
    'type': 'split',
    'direction': 'vertical',
    'size': 600,
    'sizeUnit': 'px',
    
    # Each object that is type `split` must have 2 children. 
    # These children have the keys `first` and `second` and can contain 
    # plots, text, or can be a container for more cells
    'first': {
      	
      	# Another container component ('type': 'split')
        'type': 'split',
        'direction': 'horizontal', # first and second nodes are split horizontally

        # First plot
        'first': {
            'type': 'box', 				# if the node contains a plot or text, it must be type 'box'
            'boxType': 'plot', 			# 'plot' or 'text'
            'fileId': 'chris:17674'		# URL of the plot
        },

        # Second plot
        'second': {
            'type': 'box',
            'boxType': 'plot',
            'url': 'chris:17564'
        }   
    },
    
    # every `split` container node has to contain 2 children (first and second)
    # In this case, don't render the second child by setting `boxType` to `empty`
    'second': {
        'boxType': 'empty',
        'type': 'box'
    }
  },
  'settings': {
    'title': 'Simple Example 1'
  }
}


Now let's send that dashboard description up to our plotly account. Plotly will return with a URL of the dashboard and we can always view it by visiting your [list of files in your plotly account](https://plot.ly/organize).

In [27]:
create_dashboard(dashboard_1, 'Dashboard Tutorial - Ex 2', True)

u'https://plot.ly/~chelsea_lyn/13120/'

Take a peak at the newly created dashboard! [https://plot.ly/~chelsea_lyn/13120/](https://plot.ly/~chelsea_lyn/13120/)

#### More about the JSON dashboard spec

The layout of dashboards are described in a tree where each node can have at most 2 branches. These nodes have the keys `first` and `second`. These branches may go vertically (to create a new row) or horizontally (to create a new column).


Each leaf node has a few properties:
- `boxType` - whether to render a `plot` or a `text` block. This will include more types in the future.
- `title` - the title of the block. Titles can also be supplied in the plots themselves
- `size` - the width of the container
- `sizeUnit` - `%`
- `fileId` - the plotly identifier of the plot. This is in the form `username:numeric_id`. The numeric ID can be found from the URl of the plot or from the lookup API. For example, the plot [https://plot.ly/~chris/1638](https://plot.ly/~chris/1638) has the `fileId` `chris:1638`
- `type` - Equal to `box`.


Leaf nodes are contained in container nodes that controls the orientation of the items (`horizontal` or `vertical`) and the size of the container. Container nodes have the following properties:
- `type` - Equal to `split` - this indicates the the node is a container node
- `first` - Contains one of the two leaf nodes (as described above)
- `second` - Contains the other leaf node
- `direction` - 'vertical' or 'horizontal'
- `size` - 600
- `sizeUnit` - 'px' or '%'

Finally, other dashboard settings can be found in the `settings` object. These include:
- `backgroundColor` - e.g. '#FFFFFF'
- `boxBackgroundColor` - e.g. '#ffffff'
- `boxBorderColor` - e.g. '#d8d8d8'
- `boxHeaderBackgroundColor` - e.g. '#f8f8f8'
- `foregroundColor` - e.g. '#333333'
- `headerBackgroundColor` - e.g. '#fbfbfb'
- `headerForegroundColor` - e.g. '#2b2b2b'
- `links` - a list of links in the header. set to [] to show no links. Otherwise, include links as dict with `title` and `url` as keys, e.g. [{'title': 'More Dashboard Examples', 'url': u'http://plot.ly/dashboards'}]
- `logoUrl` - an image URL to show in the header
- `title` - the title of the dashboard e.g. 'Example Dashboard'
<br>
By far the easiest way to create these dashboards programatically is to create a template with the editor and then download the JSON with the API.

Let's take a look at a more complicated example ([View the live dashboard](https://plot.ly/dashboard/chris:17750/present)).

[![Energy dashboard](https://cdn.pbrd.co/images/J8vO8r8d.png)](https://plot.ly/dashboard/chris:17750/present)
<br>
Let's download this dashboard:

In [31]:
[dashboard['filename'] for dashboard in dashboards]

[u'Tern Plots', u'Dashboard Tutorial - Energy Example']

In [32]:
# In this case, I'm interested in 'Energy Dashboard', the 3rd one that I've created 

energy_dashboard = dashboards[1]

dashboard = get(energy_dashboard['api_urls']['dashboards']).json()

# Pretty print the content
energy_dashboard_json = json.loads(dashboard['content'])
pprint.pprint(energy_dashboard_json)

{u'layout': {u'direction': u'vertical',
             u'first': {u'direction': u'vertical',
                        u'first': {u'direction': u'vertical',
                                   u'first': {u'direction': u'vertical',
                                              u'first': {u'direction': u'vertical',
                                                         u'first': {u'direction': u'vertical',
                                                                    u'first': {u'boxType': u'text',
                                                                               u'text': u'# Dashboards for a new generation of energy technology\n\nPlotly provides best-in-class data visualization tools for oil and gas firms like C12 Energy, energy software providers like Opower, and renewable energy companies like Ambri and Sighten. Plotly graphs help them quickly and easily digest large quantities of data.\n\nC-12 Energy\u2019s technology team built their real-time data acquisition, visua

I'll just re-upload this JSON description to my plotly account to create a new dashboard. In practice, you can create your dashboard templates with the editor and then download the contents or build up the JSON from scratch.

In [34]:
create_dashboard(energy_dashboard_json, 'Dashboard Tutorial - Energy Example 2', True)

u'https://plot.ly/~chelsea_lyn/13121/'

#### Reference
- [View more dashboard examples](https://plot.ly/dashboards)
- [Create dashboards online](https://plot.ly/dashboard/create)
- [Create Plotly graphs in Python](https://plot.ly/python)
- You can also embed your plotly graphs in our new [presentations editor](https://formidable.com/open-source/spectacle-editor/).
- This tutorial was written using Jupyter notebooks and our new [Jupter notebook desktop app, nteract](http://nteract.io/). You can upload and share Jupter notebooks in plotly too. Like plots and datasets, notebooks shared in plotly can be public, private, or shared privately with a secret URL.
- Creating public dashboards is free. To create private dashboards, [subscribe to a pro plan](https://plot.ly/products/cloud/).